In [66]:
#imports
import requests
import pandas as pd
import json
import time
import networkx

In [67]:
class Release_Node:
    """
    Represents a node in a networkx diagram.with basic attributes and functionalities(TBD).

    Attributes:
        release_name (str): Name of the music release.
        release_mbid (str): MusicBrainz ID.
        release_credits (list): List of credits on the music release.
    """
    def __init__(self, release_name, release_data, release_credits):
        """
        Initializes the Release_Node object.

        Args:
            release_name (str): Name of the music release.
            release_mbid (str): MusicBrainz ID.
            release_credits (list): List of credits on the music release.
        """
        self.release_name = name
        self.release_mbid = data
        self.release_credits = []

In [68]:
def search_recording(artist_name, release_title):
    """
    Retrieves all release groups for a given artist using their MusicBrainz ID.

    Args:
        artist_name (str): Name of the artist.
        release_title (str): Name of the Release

    Returns:
        list: The MBID for the release. This is the initial node for the network.
    """
    base_url = "https://musicbrainz.org/ws/2/release/"
    params = {
        "query": f"artist:\"{artist_name}\" AND release:\"{release_title}\"",
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"} # Replace with your app info

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status() # Raise an exception for bad status codes
    data = response.json()
    
    if data and data.get('releases'):
        # Return the MBID of the first matching recording
        return data['releases'][0]['id']
    return None

# Example usage:
artist = "Steely Dan"
album = "Everything Must Go"
release_mbid = search_recording(artist, album)
print(f"Recording MBID for '{album}' by '{artist}': {release_mbid}")

Recording MBID for 'Everything Must Go' by 'Steely Dan': 6c54186c-ba4c-4391-9f6f-b8521c7b11d0


In [69]:
async def get_credits_for_recording(release_mbid):
    """
    Retrieves all the credits for a given release using the releases MusicBrainz ID.

    Args:
        release_mbid (str): MusicBrainz ID for the release.

    Returns:
        list: A list of all credited personnel (including artist) on the release based on the provided release mbid.
        This list includes the name of the credit, the role of the credit, and the mbid for the credit.
    """
    base_url = f"https://musicbrainz.org/ws/2/release/{release_mbid}"
    params = {
        "inc": "artist-credits+labels+recordings+recording-level-rels+work-rels+work-level-rels+artist-rels + release-groups",  # Include artist relationships
        "fmt": "json"
    }
    headers = {"User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me)"}

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()

    credits = []
    
    if data and data.get('artist-credit'):
       for track in data['media'][0]['tracks']:
            for credit in track['recording']['relations']:
                if credit['attributes'] == []:
                    attributes = 'Not Found'
                else:
                    attributes = credit['attributes']
                credits.append((track['number'], track['title'], credit.get('artist', {}).get('name', 'Not Found'), attributes, credit.get('artist', {}).get('id', 'Not Found')))
            for credit in track['artist-credit']:
                credits.append((track['number'], track['title'], credit.get('name', 'Not Found'), credit.get('attributes', 'Not Found'), credit.get('artist', {}).get('id', 'Not Found')))
    if data['relations']:
       for credit in data['relations']:
           trackNum = 'Not Found'
           trackTitle = 'Not Found'
           if credit['attributes'] == [] or not credit['attributes']:
               attributes = 'Not Found'
           else:
               attributes = credit['attributes']
           credits.append((trackNum, trackTitle, credit.get('artist', {}).get('name', 'Not Found'), attributes, credit.get('artist', {}).get('id', 'Not Found')))

    creditNames = []
    creditList = []
    for credit in credits:
        if credit[2] not in creditNames:
            creditNames.append(credit[2])
            creditList.append((credit[2],credit[3], credit[4]))
    
    return creditList

In [70]:
def get_artist_releases(artist_mbid):
    """
    Retrieves all release groups for a given artist using their MusicBrainz ID.

    Args:
        artist_mbid (str): The MusicBrainz ID of the artist.

    Returns:
        list: A list of dictionaries, each representing a release group.
              Returns an empty list if no releases are found or an error occurs.
    """
    base_url = "https://musicbrainz.org/ws/2/release-group/"
    releases = []
    limit = 100  # Maximum results per request
    offset = 0

    while True:
        params = {
            "artist": artist_mbid,
            "fmt": "json",
            "limit": limit,
            "offset": offset
        }
        headers = {
            "User-Agent": "Mozilla/5.0 (bittah.pupil@proton.me" # Replace with your app name and contact email
        }

        try:
            response = requests.get(base_url, params=params, headers=headers)
            response.raise_for_status()  # Raise an exception for HTTP errors
            data = response.json()

            release_groups = data.get("release-groups", [])
            releases.extend(release_groups)

            # Check if there are more results to fetch
            if len(release_groups) < limit:
                break
            offset += limit

        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            break
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON response: {e}")
            break

    return releases

In [71]:
# provide initial inputs and retrieve release_mbid for the release
artist = "Steely Dan"
album = "Everything Must Go"
release_mbid = search_recording(artist, album)
print(f"Recording MBID for '{album}' by '{artist}': {release_mbid}")

# retrieve credit personnel for the initial release
if release_mbid:
    albumCredits = await get_credits_for_recording(release_mbid)
    print(albumCredits)
    print("\n\n")
else:
    print(f"Recording for '{album}' by '{artist}' not found.")

for credit in albumCredits:
    releases = get_artist_releases(credit[2])
    print(f"Credit Name: {credit[0]}:")
    print(releases)
    print("\n")
    



Recording MBID for 'Everything Must Go' by 'Steely Dan': 6c54186c-ba4c-4391-9f6f-b8521c7b11d0
[('Ted Baker', ['piano'], 'ffd99695-e69f-4d8a-9905-ab44801349a2'), ('Walter Becker', ['electric bass guitar', 'electric guitar', 'solo'], '8cc72457-488e-44e6-a5b3-d2da843b7245'), ('Keith Carlock', ['drums (drum set)'], '318c5217-5b77-46aa-8662-b7f2aa8f7774'), ('Donald Fagen', ['Rhodes piano', 'Wurlitzer electric piano'], '70047e57-0153-4117-b0fc-a1d2e322e5ef'), ('Jon Herington', ['electric guitar'], '94ad5b05-7843-44ce-b994-b17830478f7d'), ('Tony Kadleck', ['trumpet'], '24c1fe59-117b-4c66-8803-e322dc8574a4'), ('Hugh McCracken', ['electric guitar'], 'fd7ce65a-3eea-4bec-9b05-7c01d43b3b00'), ('Jim Pugh', ['trombone'], 'fd1262b1-b80c-4836-a167-0c2d7420b110'), ('Roger Rosenberg', ['baritone saxophone'], '6dd659ef-d184-405a-afb5-ed73ea27ab51'), ('Walt Weiskopf', ['alto saxophone'], '81f5f851-bf90-4e97-940a-c1e87785899c'), ('Joe Peccerillo', ['assistant'], '7cef6291-593c-498f-a2af-759424c79bb5'), ('E